Connected to Python 3.12.2

In [ ]:
#!/usr/bin/env python3
"""
PEBD Calculator per DoDFMR Volume 7A, Chapter 1 (May 2024).
With Tkinter GUI, dynamic user input, support for past/future dates up to 9999-12-31,
explicit DEP output labeled as DEP, and plain text output.
"""

import tkinter as tk
from datetime import datetime, timedelta
from tkinter import messagebox

class PEBDCalculator:
    def __init__(self, root):
        self.root = root
        self.root.title("PEBD Calculator (DoDFMR Vol 7A, Ch 1)")
        self.date_periods = []  # Stores (start, end, years, months, days, is_creditable)
        self.lost_periods = []
        self.entries = {}

        # Core date entries with updated labels
        self.create_label_entry("Date of Original Entry Armed Forces (DOEAF):", 0)
        self.create_label_entry("Initial PEBD:", 1)  # Changed from "1st Day of Active Duty"
        self.create_label_entry("End of Obligated Service Date (EOS):", 2)
        self.create_label_entry("Re-enlistment Date:", 3)  # Changed from "Reentry Date"
        tk.Label(root, text="Member Type:").grid(row=4, column=0, sticky="e", padx=5, pady=5)
        self.member_type = tk.StringVar(value="Enlisted")
        tk.OptionMenu(root, self.member_type, "Enlisted", "Officer").grid(row=4, column=1, padx=5, pady=5)

        # Buttons
        tk.Button(root, text="Add Creditable Service Period", command=self.add_service_period).grid(row=5, column=0, columnspan=2, pady=5)
        tk.Button(root, text="Add Time Lost Period", command=self.add_lost_period).grid(row=6, column=0, columnspan=2, pady=5)
        tk.Button(root, text="Add DEP Service", command=self.add_dep_service).grid(row=7, column=0, columnspan=2, pady=5)
        tk.Button(root, text="Calculate PEBD", command=self.calculate_pebd).grid(row=8, column=0, pady=10)
        tk.Button(root, text="Reset", command=self.reset).grid(row=8, column=1, pady=10)

    def create_label_entry(self, label_text, row):
        tk.Label(self.root, text=label_text).grid(row=row, column=0, sticky="e", padx=5, pady=5)
        entry = tk.Entry(self.root)
        entry.grid(row=row, column=1, padx=5, pady=5)
        self.entries[label_text.strip(":")] = entry

    def add_service_period(self):
        window = tk.Toplevel(self.root)
        window.title("Add Creditable Service Period")
        tk.Label(window, text="Start Date (YYYY-MM-DD, optional):").grid(row=0, column=0, padx=5, pady=5)
        start_entry = tk.Entry(window)
        start_entry.grid(row=0, column=1, padx=5, pady=5)
        tk.Label(window, text="End Date (YYYY-MM-DD, optional):").grid(row=1, column=0, padx=5, pady=5)
        end_entry = tk.Entry(window)
        end_entry.grid(row=1, column=1, padx=5, pady=5)
        tk.Label(window, text="OR Enter Service Period Directly:").grid(row=2, column=0, columnspan=2, pady=5)
        tk.Label(window, text="Years:").grid(row=3, column=0, padx=5, pady=5)
        years_entry = tk.Entry(window)
        years_entry.grid(row=3, column=1, padx=5, pady=5)
        tk.Label(window, text="Months:").grid(row=4, column=0, padx=5, pady=5)
        months_entry = tk.Entry(window)
        months_entry.grid(row=4, column=1, padx=5, pady=5)
        tk.Label(window, text="Days:").grid(row=5, column=0, padx=5, pady=5)
        days_entry = tk.Entry(window)
        days_entry.grid(row=5, column=1, padx=5, pady=5)
        tk.Button(window, text="Save", command=lambda: self.save_service_period(window, start_entry, end_entry, years_entry, months_entry, days_entry)).grid(row=6, column=0, columnspan=2, pady=10)

    def save_service_period(self, window, start_entry, end_entry, years_entry, months_entry, days_entry):
        start = start_entry.get().strip()
        end = end_entry.get().strip()
        years = years_entry.get().strip()
        months = months_entry.get().strip()
        days = days_entry.get().strip()

        try:
            if years or months or days:
                y = int(years) if years else 0
                m = int(months) if months else 0
                d = int(days) if days else 0
                if y < 0 or m < 0 or d < 0:
                    raise ValueError("Service periods must be non-negative.")
                self.date_periods.append((start or "N/A", end or "N/A", y, m, d, True))
            elif start and end:
                datetime.strptime(start, '%Y-%m-%d')
                datetime.strptime(end, '%Y-%m-%d')
                self.date_periods.append((start, end, 0, 0, 0, True))
            else:
                raise ValueError("Provide either dates or a service period.")
            window.destroy()
        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input: {e}. Use 'YYYY-MM-DD' for dates and integers for years/months/days.")

    def add_lost_period(self):
        window = tk.Toplevel(self.root)
        window.title("Add Time Lost Period")
        tk.Label(window, text="Start Date (YYYY-MM-DD):").grid(row=0, column=0, padx=5, pady=5)
        start_entry = tk.Entry(window)
        start_entry.grid(row=0, column=1, padx=5, pady=5)
        tk.Label(window, text="End Date (YYYY-MM-DD):").grid(row=1, column=0, padx=5, pady=5)
        end_entry = tk.Entry(window)
        end_entry.grid(row=1, column=1, padx=5, pady=5)
        tk.Button(window, text="Save", command=lambda: self.save_period(window, start_entry, end_entry, self.lost_periods)).grid(row=2, column=0, columnspan=2, pady=10)

    def save_period(self, window, start_entry, end_entry, period_list):
        start = start_entry.get()
        end = end_entry.get()
        try:
            datetime.strptime(start, '%Y-%m-%d')
            datetime.strptime(end, '%Y-%m-%d')
            period_list.append((start, end))
            window.destroy()
        except ValueError:
            messagebox.showerror("Error", "Invalid date format. Use 'YYYY-MM-DD'.")

    def add_dep_service(self):
        window = tk.Toplevel(self.root)
        window.title("Add DEP Service")
        tk.Label(window, text="DEP Start Date (YYYY-MM-DD):").grid(row=0, column=0, padx=5, pady=5)
        start_entry = tk.Entry(window)
        start_entry.grid(row=0, column=1, padx=5, pady=5)
        tk.Label(window, text="DEP End Date (YYYY-MM-DD):").grid(row=1, column=0, padx=5, pady=5)
        end_entry = tk.Entry(window)
        end_entry.grid(row=1, column=1, padx=5, pady=5)
        tk.Label(window, text="Performed IDT? (post-Nov 28, 1989):").grid(row=2, column=0, padx=5, pady=5)
        idt_var = tk.StringVar(value="No")
        tk.OptionMenu(window, idt_var, "Yes", "No").grid(row=2, column=1, padx=5, pady=5)
        tk.Button(window, text="Save", command=lambda: self.save_dep(window, start_entry, end_entry, idt_var)).grid(row=3, column=0, columnspan=2, pady=10)

    def save_dep(self, window, start_entry, end_entry, idt_var):
        start = start_entry.get()
        end = end_entry.get()
        idt = idt_var.get()
        try:
            start_dt = datetime.strptime(start, '%Y-%m-%d')
            end_dt = datetime.strptime(end, '%Y-%m-%d')
            dep_start_1985 = datetime(1985, 1, 1)
            dep_cutoff_1989 = datetime(1989, 11, 29)

            if start_dt < dep_start_1985:
                self.date_periods.append((start, end, 0, 0, 0, True))
            elif dep_start_1985 <= start_dt < dep_cutoff_1989:
                self.date_periods.append((start, end, 0, 0, 0, False))
            elif start_dt >= dep_cutoff_1989 and idt == "Yes":
                self.date_periods.append((start, end, 0, 0, 0, True))
            else:
                self.date_periods.append((start, end, 0, 0, 0, False))
            window.destroy()
        except ValueError:
            messagebox.showerror("Error", "Invalid date format. Use 'YYYY-MM-DD'.")

    def calculate_inclusive_days(self, start_date, end_date):
        d1 = datetime.strptime(start_date, '%Y-%m-%d')
        d2 = datetime.strptime(end_date, '%Y-%m-%d')
        if d2.day == 31:
            d2 = d2.replace(day=30)
        elif d2.month == 2:
            if d2.day == 29 or (d2.day == 28 and not (d2.year % 4 == 0 and (d2.year % 100 != 0 or d2.year % 400 == 0))):
                d2 = d2.replace(day=30)
        return (d2 - d1).days + 1

    def calculate_total_inclusive_days(self, periods):
        total_days = 0
        for start, end, years, months, days, is_creditable in periods:
            if is_creditable:
                if years or months or days:
                    total_days += (years * 365) + (months * 30) + days
                elif start != "N/A" and end != "N/A":
                    total_days += self.calculate_inclusive_days(start, end)
        return total_days

    def subtract_days_from_date(self, reference_date, days_to_subtract):
        ref_date = datetime.strptime(reference_date, '%Y-%m-%d')
        result_date = ref_date - timedelta(days=days_to_subtract)
        return result_date.strftime('%Y-%m-%d')

    def add_days_to_date(self, reference_date, days_to_add):
        ref_date = datetime.strptime(reference_date, '%Y-%m-%d')
        result_date = ref_date + timedelta(days=days_to_add)
        return result_date.strftime('%Y-%m-%d')

    def calculate_lost_time(self, lost_periods):
        total_lost_days = 0
        for start, end in lost_periods:
            days = self.calculate_inclusive_days(start, end)
            years = days // 360
            remaining_days = days % 360
            months = remaining_days // 30
            extra_days = remaining_days % 30
            total_lost_days += years * 360 + months * 30 + extra_days
        return total_lost_days

    def days_to_ymd(self, total_days):
        years = total_days // 365
        remaining_days = total_days % 365
        months = remaining_days // 30
        days = remaining_days % 30
        if months >= 12:
            years += months // 12
            months %= 12
        return f"{years:02d} Years, Months {months:02d}, Days {days:02d}"

    def calculate_pebd(self):
        try:
            doeaf = self.entries["Date of Original Entry Armed Forces (DOEAF)"].get()
            initial_pebd = self.entries["Initial PEBD"].get()  # Updated key
            eos = self.entries["End of Obligated Service Date (EOS)"].get()
            reenlistment_date = self.entries["Re-enlistment Date"].get()  # Updated key
            member_type = self.member_type.get()

            for date in [doeaf, initial_pebd, eos, reenlistment_date]:
                if date:
                    datetime.strptime(date, '%Y-%m-%d')

            total_service_days = self.calculate_total_inclusive_days(self.date_periods) if self.date_periods else 0
            total_lost_days = self.calculate_lost_time(self.lost_periods) if self.lost_periods else 0
            net_service_days = total_service_days - total_lost_days

            eos_dt = datetime.strptime(eos, '%Y-%m-%d')
            reenlistment_dt = datetime.strptime(reenlistment_date, '%Y-%m-%d')

            if reenlistment_dt <= eos_dt + timedelta(days=1):
                pebd = self.add_days_to_date(initial_pebd, total_lost_days) if total_lost_days > 0 and member_type == "Enlisted" else initial_pebd
                print(f"No break in service. PEBD is Initial PEBD{' adjusted for Time Lost' if total_lost_days > 0 and member_type == 'Enlisted' else ''}.")
            else:
                pebd = self.subtract_days_from_date(reenlistment_date, net_service_days)
                print(f"Break in service. PEBD = Re-enlistment Date - Net Creditable Service Days.")

            # Chronological output (descending) with plain text
            result_text = (
                f"Re-enlistment Date: {reenlistment_date}\n"
                f"Pay Entry Base Date (PEBD): {pebd}\n"
                f"EOS: {eos}\n"
                f"Initial PEBD: {initial_pebd}\n"
            )
            if self.date_periods:
                result_text += "Creditable Service Periods (including DEP if applicable):\n"
                for i, (start, end, years, months, days, is_creditable) in enumerate(self.date_periods, 1):
                    if start == "N/A" and end == "N/A":  # Non-DEP period
                        period_str = f"  Period {i}: {start} to {end} ({years:02d} Years, Months {months:02d}, Days {days:02d}{' - Not Creditable' if not is_creditable else ''})\n"
                    else:  # DEP period
                        period_str = f"  DEP: {start} to {end} ({self.days_to_ymd(self.calculate_inclusive_days(start, end))}{' - Not Creditable' if not is_creditable else ''})\n"
                    result_text += period_str
            result_text += (
                f"DOEAF: {doeaf}\n"
                f"Total service days: {self.days_to_ymd(total_service_days)}\n"
                f"Time Lost days (30-day month basis): {self.days_to_ymd(total_lost_days)}\n"
                f"Net creditable service days: {self.days_to_ymd(net_service_days)}\n"
                f"Member Type: {member_type}\n"
            )
            if self.lost_periods:
                result_text += "Time Lost Periods:\n"
                for i, (start, end) in enumerate(self.lost_periods, 1):
                    result_text += f"  Lost Period {i}: {start} to {end} ({self.days_to_ymd(self.calculate_inclusive_days(start, end))})\n"

            self.show_results(result_text)

        except ValueError as ve:
            messagebox.showerror("Error", f"Invalid date format: {ve}. Use 'YYYY-MM-DD'.")
        except Exception as e:
            messagebox.showerror("Error", f"Calculation error: {e}")

    def show_results(self, results_text):
        result_window = tk.Toplevel(self.root)
        result_window.title("PEBD Calculation Results")
        text_area = tk.Text(result_window, height=15, width=80)
        text_area.pack(padx=5, pady=5)
        text_area.insert(tk.END, results_text)
        text_area.config(state='disabled')

    def reset(self):
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.member_type.set("Enlisted")
        self.date_periods.clear()
        self.lost_periods.clear()
        print("Calculator reset for new calculation.")

if __name__ == "__main__":
    root = tk.Tk()
    app = PEBDCalculator(root)
    root.mainloop()

Break in service. PEBD = Re-enlistment Date - Net Creditable Service Days.
Calculator reset for new calculation.
